# PointNet Formal Verification with α,β-CROWN

## Safe and Verified AI - Formal Verification Project

This notebook verifies two key properties of a PointNet classifier for LiDAR point clouds:

### Property 1: Local Robustness (L∞)
For every correctly classified point x₀:
```
∀x' with ||x' - x₀||∞ ≤ ε : f(x') = f(x₀)
```
"Classification remains invariant under perturbation"

### Property 2: Safety Property
For every point x₀ classified as OBSTACLE (label=1):
```
∀x' with ||x' - x₀||∞ ≤ ε : f(x') ≠ GROUND
```
"An obstacle is NEVER misclassified as drivable ground"

### Comparison with NSGA-III SLAM Attack
| Perturbation | SLAM Error | Status |
|--------------|------------|--------|
| 0 cm | 23 cm | Baseline |
| 1.5 cm | 32 cm | Degradation starts |
| 4.6 cm | 85 cm | SLAM fails |
| 20 cm | - | Complete failure |

## 1. Setup and Installation

In [ ]:
# Check GPU
!nvidia-smi

In [ ]:
# Install dependencies
!pip install torch numpy onnx onnxruntime pyyaml packaging appdirs gurobipy sortedcontainers -q
!git clone https://github.com/Verified-Intelligence/alpha-beta-CROWN.git 2>/dev/null || true
!cd /content/alpha-beta-CROWN && git submodule update --init --recursive

In [ ]:
# Setup paths and imports
import sys
sys.path.insert(0, '/content/alpha-beta-CROWN/auto_LiRPA')
sys.path.insert(0, '/content/alpha-beta-CROWN/complete_verifier')

# Mock onnxoptimizer to avoid compilation
from types import ModuleType
mock = ModuleType('onnxoptimizer')
mock.optimize = lambda model, passes=None: model
sys.modules['onnxoptimizer'] = mock

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import json
from datetime import datetime

print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Import alpha-beta-CROWN
from api import ABCrownSolver, ConfigBuilder, VerificationSpec, input_vars, output_vars
print("α,β-CROWN imported successfully!")

## 2. Model Definitions

In [ ]:
class PointNetForVerification(nn.Module):
    """
    PointNet Base (~100k parameters)
    Based on: Qi et al., "PointNet: Deep Learning on Point Sets" (CVPR 2017)
    """
    def __init__(self, num_points=64, num_classes=2, use_tnet=True, pooling="max"):
        super().__init__()
        self.num_points = num_points
        self.num_classes = num_classes
        self.use_tnet = use_tnet
        self.pooling = pooling

        if use_tnet:
            self.tnet_conv1 = nn.Conv1d(3, 64, 1)
            self.tnet_conv2 = nn.Conv1d(64, 128, 1)
            self.tnet_conv3 = nn.Conv1d(128, 256, 1)
            self.tnet_fc1 = nn.Linear(256, 128)
            self.tnet_fc2 = nn.Linear(128, 64)
            self.tnet_fc3 = nn.Linear(64, 9)
            self.tnet_fc3.weight.data.zero_()
            self.tnet_fc3.bias.data.copy_(torch.eye(3).view(-1))

        self.conv1 = nn.Conv1d(3, 64, 1)
        self.conv2 = nn.Conv1d(64, 128, 1)
        self.conv3 = nn.Conv1d(128, 256, 1)
        self.fc1 = nn.Linear(256, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, num_classes)

    def forward(self, x):
        batch_size = x.shape[0]
        if x.dim() == 2:
            x = x.view(batch_size, self.num_points, 3)

        if self.use_tnet:
            x_t = x.transpose(1, 2)
            t = F.relu(self.tnet_conv1(x_t))
            t = F.relu(self.tnet_conv2(t))
            t = F.relu(self.tnet_conv3(t))
            t = torch.max(t, dim=2)[0] if self.pooling == "max" else torch.mean(t, dim=2)
            t = F.relu(self.tnet_fc1(t))
            t = F.relu(self.tnet_fc2(t))
            t = self.tnet_fc3(t).view(batch_size, 3, 3)
            x = torch.bmm(x, t)

        x = x.transpose(1, 2)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = torch.max(x, dim=2)[0] if self.pooling == "max" else torch.mean(x, dim=2)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)


class PointNetLarge(nn.Module):
    """
    PointNet Large (~1.5M parameters)
    Deeper architecture for more robust verification testing.
    """
    def __init__(self, num_points=64, num_classes=2, use_tnet=True, pooling="max"):
        super().__init__()
        self.num_points = num_points
        self.num_classes = num_classes
        self.use_tnet = use_tnet
        self.pooling = pooling

        if use_tnet:
            self.tnet_conv1 = nn.Conv1d(3, 128, 1)
            self.tnet_conv2 = nn.Conv1d(128, 256, 1)
            self.tnet_conv3 = nn.Conv1d(256, 512, 1)
            self.tnet_fc1 = nn.Linear(512, 256)
            self.tnet_fc2 = nn.Linear(256, 128)
            self.tnet_fc3 = nn.Linear(128, 9)
            self.tnet_fc3.weight.data.zero_()
            self.tnet_fc3.bias.data.copy_(torch.eye(3).view(-1))

        self.conv1 = nn.Conv1d(3, 128, 1)
        self.conv2 = nn.Conv1d(128, 256, 1)
        self.conv3 = nn.Conv1d(256, 512, 1)
        self.conv4 = nn.Conv1d(512, 1024, 1)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, num_classes)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        batch_size = x.shape[0]
        if x.dim() == 2:
            x = x.view(batch_size, self.num_points, 3)

        if self.use_tnet:
            x_t = x.transpose(1, 2)
            t = F.relu(self.tnet_conv1(x_t))
            t = F.relu(self.tnet_conv2(t))
            t = F.relu(self.tnet_conv3(t))
            t = torch.max(t, dim=2)[0] if self.pooling == "max" else torch.mean(t, dim=2)
            t = F.relu(self.tnet_fc1(t))
            t = F.relu(self.tnet_fc2(t))
            t = self.tnet_fc3(t).view(batch_size, 3, 3)
            x = torch.bmm(x, t)

        x = x.transpose(1, 2)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = torch.max(x, dim=2)[0] if self.pooling == "max" else torch.mean(x, dim=2)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        return self.fc4(x)


# Count parameters
model_base = PointNetForVerification()
model_large = PointNetLarge()
print(f"PointNet Base:  {sum(p.numel() for p in model_base.parameters()):,} parameters")
print(f"PointNet Large: {sum(p.numel() for p in model_large.parameters()):,} parameters")

## 3. Load Data and Model

**Upload the following files to `/content/`:**
- `pointnet.pth` (trained model)
- `test_groups.npy` (test point clouds)
- `test_labels.npy` (test labels)

In [ ]:
import os
import shutil

# Create folders
os.makedirs('models', exist_ok=True)
os.makedirs('data/pointnet', exist_ok=True)

# Copy files
if os.path.exists('/content/pointnet.pth'):
    shutil.copy('/content/pointnet.pth', 'models/pointnet.pth')
if os.path.exists('/content/test_groups.npy'):
    shutil.copy('/content/test_groups.npy', 'data/pointnet/test_groups.npy')
if os.path.exists('/content/test_labels.npy'):
    shutil.copy('/content/test_labels.npy', 'data/pointnet/test_labels.npy')

print("Files:")
!ls -la models/
!ls -la data/pointnet/

In [ ]:
# Load model
MODEL_TYPE = "base"  # Change to "large" after training large model

if MODEL_TYPE == "large":
    model = PointNetLarge(num_points=64, num_classes=2, use_tnet=True, pooling="max")
else:
    model = PointNetForVerification(num_points=64, num_classes=2, use_tnet=True, pooling="max")

checkpoint = torch.load('models/pointnet.pth', map_location='cpu', weights_only=True)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

# Load data
X_test = np.load('data/pointnet/test_groups.npy')
y_test = np.load('data/pointnet/test_labels.npy')

print(f"Model: {MODEL_TYPE.upper()} ({sum(p.numel() for p in model.parameters()):,} params)")
print(f"Test samples: {len(X_test)}")
print(f"Labels distribution: GROUND={sum(y_test==0)}, OBSTACLE={sum(y_test==1)}")

## 4. Verification Functions

In [ ]:
def verify_robustness(model, sample, label, epsilon, timeout=120):
    """
    Property 1: Local Robustness
    Verify that classification does not change under L-inf perturbation.
    """
    sample_flat = sample.T.flatten() if sample.shape == (3, 64) else sample.flatten()
    x = input_vars(192)
    y = output_vars(2)

    input_constraint = (x >= sample_flat - epsilon) & (x <= sample_flat + epsilon)
    output_constraint = y[label] > y[1 - label]

    spec = VerificationSpec.build_spec(
        input_vars=x, output_vars=y,
        input_constraint=input_constraint,
        output_constraint=output_constraint,
    )

    cfg = ConfigBuilder.from_defaults().set(
        bab__timeout=timeout,
        general__enable_incomplete_verification=True,
        general__complete_verifier="bab",
        general__conv_mode="matrix",
        attack__pgd_order="before",
        attack__pgd_steps=100,
        attack__pgd_restarts=30,
    )

    solver = ABCrownSolver(spec, model, config=cfg)
    result = solver.solve()
    verified = result.status in ["safe", "safe-incomplete", "verified", "unsat"]
    return {"status": result.status, "verified": verified}


def verify_safety(model, sample, epsilon, timeout=120):
    """
    Property 2: Safety Property
    For OBSTACLE samples: verify that no perturbation causes GROUND classification.
    """
    sample_flat = sample.T.flatten() if sample.shape == (3, 64) else sample.flatten()
    x = input_vars(192)
    y = output_vars(2)

    # Input: perturbed OBSTACLE
    input_constraint = (x >= sample_flat - epsilon) & (x <= sample_flat + epsilon)
    # Output: OBSTACLE (y[1]) > GROUND (y[0]) - i.e., never classified as GROUND
    output_constraint = y[1] > y[0]

    spec = VerificationSpec.build_spec(
        input_vars=x, output_vars=y,
        input_constraint=input_constraint,
        output_constraint=output_constraint,
    )

    cfg = ConfigBuilder.from_defaults().set(
        bab__timeout=timeout,
        general__enable_incomplete_verification=True,
        general__complete_verifier="bab",
        general__conv_mode="matrix",
        attack__pgd_order="before",
        attack__pgd_steps=100,
        attack__pgd_restarts=30,
    )

    solver = ABCrownSolver(spec, model, config=cfg)
    result = solver.solve()
    verified = result.status in ["safe", "safe-incomplete", "verified", "unsat"]
    return {"status": result.status, "verified": verified}


print("Verification functions defined!")

## 5. Property 1: Local Robustness Verification

Test epsilon from 0.5cm to 20cm

In [ ]:
# Property 1: Robustness
# Epsilon values: 0.5cm to 20cm
epsilons_robustness = [0.003, 0.005, 0.01, 0.02, 0.03, 0.05, 0.07, 0.10, 0.13]
# 0.003 ~ 0.5cm, 0.005 ~ 0.75cm, 0.01 ~ 1.5cm, 0.02 ~ 3cm, 0.03 ~ 4.5cm
# 0.05 ~ 7.5cm, 0.07 ~ 10.5cm, 0.10 ~ 15cm, 0.13 ~ 20cm

n_samples = min(10, len(X_test))
robustness_results = {}

print("="*70)
print("PROPERTY 1: LOCAL ROBUSTNESS (L∞)")
print("Verifying: ∀x' with ||x' - x₀||∞ ≤ ε : f(x') = f(x₀)")
print("="*70)

for eps in epsilons_robustness:
    cm = eps * 150
    print(f"\nε = {eps} (~{cm:.1f} cm)")
    print("-"*40)

    verified_count = 0
    unsafe_count = 0
    timeout_count = 0
    sample_details = []

    for i in range(n_samples):
        sample = X_test[i]
        label = int(y_test[i])
        label_str = "GROUND" if label == 0 else "OBSTACLE"

        try:
            result = verify_robustness(model, sample, label, eps, timeout=120)
            if result["verified"]:
                verified_count += 1
                status = "✓ VERIFIED"
            elif "timeout" in result["status"]:
                timeout_count += 1
                status = "⏱ TIMEOUT"
            else:
                unsafe_count += 1
                status = f"✗ UNSAFE ({result['status']})"
            sample_details.append({"sample": i, "label": label_str, "status": result["status"], "verified": result["verified"]})
        except Exception as e:
            timeout_count += 1
            status = f"⚠ ERROR"
            sample_details.append({"sample": i, "label": label_str, "status": "error", "verified": False})

        print(f"  Sample {i} ({label_str:8}): {status}")

    robustness_results[str(eps)] = {
        "epsilon": eps,
        "cm": cm,
        "verified": verified_count,
        "unsafe": unsafe_count,
        "timeout": timeout_count,
        "verified_pct": 100 * verified_count / n_samples,
        "samples": sample_details
    }

    print(f"  Summary: {verified_count}/{n_samples} verified ({100*verified_count/n_samples:.0f}%)")

## 6. Property 2: Safety Property Verification

Only for OBSTACLE samples: verify they never become GROUND

In [ ]:
# Property 2: Safety
# Same epsilon range
epsilons_safety = [0.003, 0.005, 0.01, 0.02, 0.03, 0.05, 0.07, 0.10, 0.13]

# Get only OBSTACLE samples
obstacle_indices = np.where(y_test == 1)[0]
n_obstacle_samples = min(10, len(obstacle_indices))

safety_results = {}

print("="*70)
print("PROPERTY 2: SAFETY PROPERTY")
print("Verifying: ∀x' with ||x' - x₀||∞ ≤ ε ∧ f(x₀)=OBSTACLE : f(x') ≠ GROUND")
print(f"Testing on {n_obstacle_samples} OBSTACLE samples")
print("="*70)

for eps in epsilons_safety:
    cm = eps * 150
    print(f"\nε = {eps} (~{cm:.1f} cm)")
    print("-"*40)

    verified_count = 0
    unsafe_count = 0
    timeout_count = 0
    sample_details = []

    for idx in obstacle_indices[:n_obstacle_samples]:
        sample = X_test[idx]

        try:
            result = verify_safety(model, sample, eps, timeout=120)
            if result["verified"]:
                verified_count += 1
                status = "✓ SAFE (never GROUND)"
            elif "timeout" in result["status"]:
                timeout_count += 1
                status = "⏱ TIMEOUT"
            else:
                unsafe_count += 1
                status = f"☠ UNSAFE - CAN BECOME GROUND!"
            sample_details.append({"sample": int(idx), "status": result["status"], "verified": result["verified"]})
        except Exception as e:
            timeout_count += 1
            status = f"⚠ ERROR"
            sample_details.append({"sample": int(idx), "status": "error", "verified": False})

        print(f"  Sample {idx} (OBSTACLE): {status}")

    safety_results[str(eps)] = {
        "epsilon": eps,
        "cm": cm,
        "verified": verified_count,
        "unsafe": unsafe_count,
        "timeout": timeout_count,
        "verified_pct": 100 * verified_count / n_obstacle_samples if n_obstacle_samples > 0 else 0,
        "samples": sample_details
    }

    safety_status = "✅ VERIFIED" if unsafe_count == 0 else "❌ VIOLATED"
    print(f"  Safety: {safety_status} ({verified_count}/{n_obstacle_samples} samples safe)")

## 7. Results Summary

In [ ]:
print("\n" + "="*70)
print("FINAL RESULTS SUMMARY")
print("="*70)

# Property 1 Table
print("\n### PROPERTY 1: LOCAL ROBUSTNESS ###")
print(f"{'Epsilon':>8} | {'~cm':>6} | {'Verified':>10} | {'Unsafe':>10} | {'Timeout':>10}")
print("-"*55)
for eps_str, r in robustness_results.items():
    print(f"{float(eps_str):>8.3f} | {r['cm']:>5.1f}  | {r['verified_pct']:>8.0f}%  | {100*r['unsafe']/n_samples:>8.0f}%  | {100*r['timeout']/n_samples:>8.0f}%")

# Property 2 Table
print("\n### PROPERTY 2: SAFETY (OBSTACLE → never GROUND) ###")
print(f"{'Epsilon':>8} | {'~cm':>6} | {'Safe':>10} | {'Unsafe':>10} | {'Status':>15}")
print("-"*60)
for eps_str, r in safety_results.items():
    status = "✅ VERIFIED" if r['unsafe'] == 0 and r['timeout'] == 0 else ("⚠ INCOMPLETE" if r['timeout'] > 0 else "❌ VIOLATED")
    print(f"{float(eps_str):>8.3f} | {r['cm']:>5.1f}  | {r['verified_pct']:>8.0f}%  | {100*r['unsafe']/n_obstacle_samples:>8.0f}%  | {status:>15}")

# Comparison with SLAM
print("\n" + "="*70)
print("COMPARISON WITH NSGA-III SLAM ATTACK")
print("="*70)
print("""
| Perturbation | SLAM Error | Formal Verification |
|--------------|------------|---------------------|
| 0 cm         | 23 cm      | Baseline            |
| 1.5 cm       | 32 cm      | Check ε=0.01 above  |
| 3.0 cm       | ~50 cm     | Check ε=0.02 above  |
| 4.5 cm       | ~70 cm     | Check ε=0.03 above  |
| 7.5 cm       | -          | Check ε=0.05 above  |
| 15 cm        | -          | Check ε=0.10 above  |
| 20 cm        | -          | Check ε=0.13 above  |
""")

In [ ]:
# Save results
final_results = {
    "timestamp": datetime.now().isoformat(),
    "model": MODEL_TYPE,
    "n_samples": n_samples,
    "n_obstacle_samples": n_obstacle_samples,
    "property1_robustness": robustness_results,
    "property2_safety": safety_results,
}

with open('verification_complete_results.json', 'w') as f:
    json.dump(final_results, f, indent=2)

print("Results saved to verification_complete_results.json")

# Download
from google.colab import files
files.download('verification_complete_results.json')

## 8. Visualizations

In [ ]:
import matplotlib.pyplot as plt

# Extract data for plotting
eps_cm = [r['cm'] for r in robustness_results.values()]
robustness_pct = [r['verified_pct'] for r in robustness_results.values()]
safety_pct = [r['verified_pct'] for r in safety_results.values()]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Robustness
ax1 = axes[0]
ax1.plot(eps_cm, robustness_pct, 'b-o', linewidth=2, markersize=8, label='Robustness')
ax1.axhline(y=50, color='r', linestyle='--', label='50% threshold')
ax1.axvline(x=1.5, color='orange', linestyle=':', label='SLAM degradation (1.5cm)')
ax1.axvline(x=4.6, color='red', linestyle=':', label='SLAM failure (4.6cm)')
ax1.set_xlabel('Perturbation (cm)', fontsize=12)
ax1.set_ylabel('Verified (%)', fontsize=12)
ax1.set_title('Property 1: Local Robustness', fontsize=14)
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_ylim([0, 105])

# Plot 2: Safety
ax2 = axes[1]
colors = ['green' if p == 100 else ('orange' if p > 50 else 'red') for p in safety_pct]
ax2.bar(eps_cm, safety_pct, color=colors, alpha=0.7, width=0.8)
ax2.axhline(y=100, color='green', linestyle='-', linewidth=2, label='Safety verified')
ax2.axvline(x=1.5, color='orange', linestyle=':', label='SLAM degradation')
ax2.axvline(x=4.6, color='red', linestyle=':', label='SLAM failure')
ax2.set_xlabel('Perturbation (cm)', fontsize=12)
ax2.set_ylabel('Safe Samples (%)', fontsize=12)
ax2.set_title('Property 2: Safety (OBSTACLE → never GROUND)', fontsize=14)
ax2.legend()
ax2.grid(True, alpha=0.3, axis='y')
ax2.set_ylim([0, 105])

plt.tight_layout()
plt.savefig('verification_results.png', dpi=150, bbox_inches='tight')
plt.show()

files.download('verification_results.png')